In [1]:
import json
from bs4 import BeautifulSoup as bs
import requests
import pprint
import time
import random

#### 1) Необходимо собрать информацию о вакансиях на должность программиста или разработчика с сайта job.ru или hh.ru. (Можно с обоих сразу) Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе:   
* Наименование вакансии,  
* Предлагаемую зарплату,  
* Ссылку на саму вакансию  
  
#### 2) Доработать приложение таким образом, чтобы можно было искать разработчиков на разные языки программирования (Например Python, Java, C++)

# Конфиг:

In [2]:
# Укажите здесь вакансию которую вы ищете
JOB_NAME = "программист с++"

# Укажите сколько страниц с вакансиями достаточно пропарсить 
# (если страниц с результатами будет меньше чем заданное число, то будут пропарсены все страницы):
pages_enough = 3

# Искать только вакансии с указанными зарплатами (если false, ищутся все вакансии):
only_with_salary=False

In [3]:
headers = requests.utils.default_headers()
headers['User-Agent'] = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.101 Safari/537.36"

In [4]:
def make_link_for_jobname(jobname_string, only_with_salary=False):
    jobname_string = jobname_string.replace("+", "%2B")
    jobname_string = jobname_string.replace("#", "%23")
    jobname_split = jobname_string.split(" ")
    only_with_salary = "true" if only_with_salary else "false"
    link = "https://hh.ru/search/vacancy?L_is_autosearch=false&area=1&only_with_salary=%s&clusters=true&enable_snippets=true&text=%s&page=0" % (only_with_salary, "+".join(jobname_split))
    return link

In [5]:
def find_next_page_link(page_content):
    if page_content.find('a', {"class": "bloko-button HH-Pager-Controls-Next HH-Pager-Control"}, href=True) is None:
        return False
    else:
        return "https://hh.ru" + page_content.find('a', {"class": "bloko-button HH-Pager-Controls-Next HH-Pager-Control"}, href=True)['href']

In [6]:
def find_all_vacancies_items(parsed_html):
    return parsed_html.find_all('div', {'class': 'vacancy-serp-item'})

In [7]:
def parse_vacancy_item(vacancy_item, debug=False):
    
    vacancy_item_second_child = vacancy_item.findChild().find_next_sibling()
    vacancy_item_third_child = vacancy_item.findChild().find_next_sibling().find_next_sibling()

    tmp_jobname = vacancy_item_second_child.findChild().getText()
    if debug:
        print("=============================================================================")
        print("Название:\t", tmp_jobname, flush=True)
        
    tmp_vacancy_href = vacancy_item_second_child.findChild(href=True)['href']
    if debug:
        print("Ссылка:\t", tmp_vacancy_href, flush=True)
        
    tmp_value = vacancy_item_second_child.findChild().find_next_sibling().getText()
    tmp_value = tmp_value if tmp_value != "" else "Не указана"
    if debug:
        print("Зарплата:\t", tmp_value, flush=True)
        
    tmp_obyazan = vacancy_item.find("div", {"data-qa": "vacancy-serp__vacancy_snippet_responsibility"}).getText()
    if debug:
        print("Обязанности:\t", tmp_obyazan, flush=True)
        
    tmp_requir = vacancy_item.find("div", {"data-qa": "vacancy-serp__vacancy_snippet_requirement"}).getText()
    if debug:
        print("Требования:\t", tmp_requir, flush=True)
        print("=============================================================================")

    return {"job": tmp_jobname, "link": tmp_vacancy_href, "value": tmp_value, "respons": tmp_obyazan, "req": tmp_requir}

In [8]:
link = make_link_for_jobname(JOB_NAME, only_with_salary=only_with_salary)
all_job_offers = []
pages_counter = 0

while link:
#     print(link)
    get_ = requests.get(link, headers=headers)
    get_.encoding = 'utf-8'
    html = get_.text
    parsed_html = bs(html,'lxml')
    link = find_next_page_link(parsed_html)
    all_vacancies = find_all_vacancies_items(parsed_html)
    for vacancy_item in all_vacancies:
        all_job_offers.append(parse_vacancy_item(vacancy_item))
    pages_counter += 1
    print("\r#%d/%d страниц пропарсено" % (pages_counter, pages_enough), end = "", flush=True)
    if pages_counter == pages_enough:
        link = False
print()

#3/3 страниц пропарсено


In [9]:
with open("vacancies.txt", "wt") as output_file:
    if len(all_job_offers) > 0:
        for rec in all_job_offers:
            #{"job": tmp_jobname, "link": tmp_vacancy_href, "value": tmp_value, "respons": tmp_obyazan, "req": tmp_requir}
            output_file.write("Вакансия:\n\t" + rec["job"] + "\n")
            output_file.write("Зарплата:\n\t" + rec["value"] + "\n")
            output_file.write("Ссылка:\n\t" + rec["link"] + "\n")
            output_file.write("Обязанности:\n\t" + rec["respons"] + "\n")
            output_file.write("Требования:\n\t" + rec["req"] + "\n")
            output_file.write("=====================================================================================\n\n")
    else:
        output_file.write("Вакансий по запросу \"%s\" не найдено.\n" % JOB_NAME)
            

In [10]:
with open("vacancies.txt", "rt") as file:
    print(file.read())

Вакансия:
	С++ программист
Зарплата:
	130 000-210 000 руб.
Ссылка:
	https://tomsk.hh.ru/vacancy/32306428?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D1%81%2B%2B
Обязанности:
	Развитие и поддержка одного из основных продуктов компании - «Интеллект».
Требования:
	Использование С++ в качестве основного рабочего инструмента в течении последних 3 лет, желательно с опытом использования Microsoft Visual Studio. 

Вакансия:
	Программист С++
Зарплата:
	до 160 000 руб.
Ссылка:
	https://tomsk.hh.ru/vacancy/32304535?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20%D1%81%2B%2B
Обязанности:
	Разработка модулей обработчика сетевых пакетов. Исследование и оптимизация производительности приложения. Участие в code review, контроль качества кода. Работа в команде...
Требования:
	Отличное знание современного C++. Умение писать понятный поддерживаемый код. Умение работать с Linux, Git, CMake. Хорошее знание сетевых протоколов (TCP...

Вакансия:
	Пр